In [3]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
all_data = pd.read_csv("crx.data", header = None)
all_data.replace('?', np.NaN, inplace=True)

data_train, data_test, labels_train, labels_test = train_test_split(all_data.iloc[:,:15], all_data.iloc[:,[15]], test_size=0.8, random_state=78)

imp = SimpleImputer(missing_values=np.NaN, strategy="most_frequent")
cat_data_train = list ()
num_data_train = list ()
cat_data_test = list ()
num_data_test = list ()
cat_cols = [1,4,5,6,7,9,10,12,13]
for col in cat_cols :
  cat_data_train.append(pd.DataFrame(imp.fit_transform(data_train.iloc[:,[col-1]])))
  cat_data_test.append(pd.DataFrame(imp.transform(data_test.iloc[:,[col-1]])))
num_cols = [2,3,8,11,14,15]
imp = SimpleImputer(missing_values=np.NaN, strategy="mean")
for col in num_cols :
  num_data_train.append(pd.DataFrame(imp.fit_transform(data_train.iloc[:,[col-1]])))
  num_data_test.append(pd.DataFrame(imp.transform(data_test.iloc[:,[col-1]])))

j1 , j2 = 0, 0

idata_train = pd.DataFrame()
idata_test = pd.DataFrame()
for col in range(1,16) :
  if j2 < 9 and cat_cols[j2] == col :
    idata_train = pd.concat([idata_train,cat_data_train[j2]],join="outer",axis=1)
    idata_test = pd.concat([idata_test, cat_data_test[j2]],join="outer",axis=1)
    j2 = j2 + 1
  else :
    idata_train = pd.concat([idata_train,num_data_train[j1]],join="outer",axis=1)
    idata_test = pd.concat([idata_test,num_data_test[j1]],join="outer",axis=1)    
    j1 = j1 + 1

idata_train.columns = list(range(1,16))
idata_test.columns = list(range(1,16))

# NEW CODE HERE -- NOTE : columns must be 46 and that is what the output shows
temp = pd.concat([idata_train,idata_test], join="outer", axis=0)
dummy_data = pd.get_dummies(temp, [0,3,4,5,6,8,9,11,12])
np_dummy_data = dummy_data.apply(pd.to_numeric).values

np_data_train = np_dummy_data[:138,:] #οι γραμμές δεν αλλάζουν θέση οπότε αφού σειριακά
np_data_test = np_dummy_data[138:,:]  #ενώσαμε τα train, test σειριακά θα τα χωρίσουμε

mapping = {'-' : 0, '+' : 1}
labels_train[15] = labels_train[15].map(mapping)
labels_test[15] = labels_test[15].map(mapping)

np_labels_train = labels_train.values.flatten()
np_labels_test = labels_test.values.flatten()

In [4]:
# ΑΠΟΤΕΛΕΣΜΑΤΑ ΧΩΡΙΣ PREPROCESSING
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import precision_recall_fscore_support
gnb = GaussianNB()
gnb_model = gnb.fit(np_data_train, np_labels_train)
gnb_pred = gnb_model.predict(np_data_test)
gnb_conf_matrix = confusion_matrix(np_labels_test, gnb_pred)
print("Confusion Maxtrix : \n", gnb_conf_matrix)
print("f1-micro average : ", f1_score(np_labels_test, gnb_pred, average = 'micro'))
print("f1-macro average : ", f1_score(np_labels_test, gnb_pred, average = 'macro'))
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='micro'), "\n")
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='macro'), "\n")
#print(classification_report(np_labels_test, gnb_pred))

Confusion Maxtrix : 
 [[278  31]
 [ 63 180]]
f1-micro average :  0.8297101449275363
f1-macro average :  0.8241680786174178


In [5]:
from imblearn.pipeline import Pipeline

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

selector = VarianceThreshold()
scaler = MinMaxScaler()
pca = PCA()
gnb = GaussianNB()
pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('gnb', gnb)])
pipe.fit(np_data_train, np_labels_train)
preds = pipe.predict(np_data_test)
print("Confusion Matrix : \n", confusion_matrix(np_labels_test, preds))
print("f1-micro average : ", f1_score(np_labels_test, preds, average = 'micro'))
print("f1-macro average : ", f1_score(np_labels_test, preds, average = 'macro'))
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='micro'), "\n")
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='macro'), "\n")
#print(classification_report(np_labels_test, preds))

Confusion Matrix : 
 [[259  50]
 [ 71 172]]
f1-micro average :  0.7807971014492753
f1-macro average :  0.7752132868897976


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
train_var = np_data_train.var(axis=0)
print(train_var)
print(np.max(train_var))

[1.22845600e+02 2.56495069e+01 1.40409780e+01 2.20494119e+01
 5.97578133e+04 3.63519072e+06 2.24585171e-01 2.24585171e-01
 0.00000000e+00 1.61730729e-01 1.61730729e-01 1.61730729e-01
 0.00000000e+00 1.61730729e-01 9.68809074e-02 1.61730729e-01
 7.33564377e-02 3.49191346e-02 3.49191346e-02 4.15879017e-02
 7.93950851e-02 7.19386683e-03 7.93950851e-02 7.33564377e-02
 1.08013022e-01 0.00000000e+00 6.72127704e-02 3.49191346e-02
 7.33564377e-02 0.00000000e+00 4.15879017e-02 1.57372401e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.22222222e-01
 1.42827137e-02 2.43646293e-01 2.43646293e-01 2.47427011e-01
 2.47427011e-01 2.49527410e-01 2.49527410e-01 8.53287125e-02
 7.19386683e-03 7.93950851e-02]
3635190.721277043


In [15]:
#ΑΠΟΤΕΛΕΣΜΑΤΑ ΒΕΛΤΙΣΤΟΥ PIPE
from sklearn.model_selection import GridSearchCV

vthreshold = [0, 1e-2, 0.05, 0.08, 1e-1, 0.2, 0.3, 0.4, 0.5, 1, 1e+1]
n_components = [1,2,3,4,5,6,7,8,9]
pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('gnb', gnb)], memory = 'tmp')
estimator = GridSearchCV(pipe, dict(selector__threshold=vthreshold, pca__n_components=n_components), cv=10, scoring='f1_macro',n_jobs=-1)

In [16]:
import time
start_time = time.time()
estimator.fit(np_data_train, np_labels_train)
preds = estimator.predict(np_data_test)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time()-start_time))
print("Confusion Matrix : \n", confusion_matrix(np_labels_test, preds))
print("f1-micro average : ", f1_score(np_labels_test, preds, average = 'micro'))
print("f1-macro average : ", f1_score(np_labels_test, preds, average = 'macro'))
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='micro'), "\n")
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='macro'), "\n")
#print(classification_report(np_labels_test, preds))

Συνολικός χρόνος fit και predict: 6.610998153686523 seconds
Confusion Matrix : 
 [[282  27]
 [ 60 183]]
f1-micro average :  0.8423913043478262
f1-macro average :  0.8371532334360791


In [12]:
print(estimator.best_estimator_)
print(estimator.best_params_)

Pipeline(memory='tmp',
         steps=[('selector', VarianceThreshold(threshold=0.1)),
                ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('pca',
                 PCA(copy=True, iterated_power='auto', n_components=5,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('gnb', GaussianNB(priors=None, var_smoothing=1e-09))],
         verbose=False)
{'pca__n_components': 5, 'selector__threshold': 0.1}


In [9]:
#ΑΠΟΤΕΛΕΣΜΑΤΑ PIPE ΧΩΡΙΣ SCALER ΑΛΛΑ ΔΕΝ ΒΕΛΤΙΩΝΟΥΝ ΤΑ ΑΠΟΤΕΛΕΣΜΑΤΑ
pipe = Pipeline(steps=[('selector', selector), ('pca', pca), ('gnb', gnb)], memory = 'tmp')
estimator = GridSearchCV(pipe, dict(selector__threshold=vthreshold, pca__n_components=n_components), cv=10, scoring='f1_macro',n_jobs=-1)
estimator.fit(np_data_train, np_labels_train)
preds = estimator.predict(np_data_test)
print("Confusion Matrix : \n", confusion_matrix(np_labels_test, preds))
print("f1-micro average : ", f1_score(np_labels_test, preds, average = 'micro'))
print("f1-macro average : ", f1_score(np_labels_test, preds, average = 'macro'))
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='micro'), "\n")
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='macro'), "\n")
#print(classification_report(np_labels_test, preds))
print(estimator.best_estimator_)
print(estimator.best_params_)

Confusion Matrix : 
 [[278  31]
 [ 92 151]]
f1-micro average :  0.7771739130434783
f1-macro average :  0.7647197435675299
Pipeline(memory='tmp',
         steps=[('selector', VarianceThreshold(threshold=0)),
                ('pca',
                 PCA(copy=True, iterated_power='auto', n_components=9,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('gnb', GaussianNB(priors=None, var_smoothing=1e-09))],
         verbose=False)
{'pca__n_components': 9, 'selector__threshold': 0}
